In [1]:
%pwd


'c:\\Users\\pc\\Desktop\\End-to-End-Mlops\\End-to-End-MLOPS\\research'

In [14]:
import sys
import os
sys.path.append(os.path.abspath("src"))


In [3]:
%pwd

'c:\\Users\\pc\\Desktop\\End-to-End-Mlops\\End-to-End-MLOPS\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\pc\\Desktop\\End-to-End-Mlops\\End-to-End-MLOPS'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [21]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories


In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
            
        )
        return data_ingestion_config
        
            
        

In [26]:
import os
import urllib.request as request
import zipfile
from src.mlProject.utils import logger
from mlProject.utils.common import get_size


In [30]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
            url = self.config.source_URL,
            filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info \n{headers}")    
        else:
            logger.info(f"file already eists of size:{get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
                

In [31]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e  

[2025-05-18 13:50:17,581: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-18 13:50:17,584: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-18 13:50:17,588: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-18 13:50:17,590: INFO: common: created directory at: artifacts]
[2025-05-18 13:50:17,592: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-18 13:50:17,595: INFO: 340460481: file already eists of size:~23 KB]
